# Code for File Conversion to Automation Formats

In [18]:
# Import libraries
import csv
import os
import openpyxl
import serial

### Write to XLS (for Hamilton)

**Comment:** We are using a Hamilton NIMBUS with staggered input/outputs, so we define some reordering.

In [ ]:
def contains_96_reactions(filename):
    '''
    Function to see if the file contains 96 reactions

    @param file : the file to read
    @return bool : true or false if file exists
    '''
    # Counter set to 0
    count = 0
    # Try importing the file
    try:
        with open(filename, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                count += 1
    
    except:
        print('ERROR: NO FILE EXISTS')
        return False

    if count == 96:
        return True

    else:
        return False

def named_association(names, values):
    '''
    Function to create dictionary for wells and concentrations

    @param names : names of the wells
    @param values : volumes to dispense
    @return associations : dictionary of key-value pairs
    '''
    # Initialize dictionary
    associations = {}

    # Iterate names
    for name, value in zip(names, values):

        # Set name to volume
        associations[name] = value

    # return dictionary
    return associations

def convert_CSV_to_NIMBUS(filename):
    '''
    Function to convert CSV format to NIMBUS and export it

    @param filename : name fo the file to convert
    '''
    # Default order
    default_well_pos = [''.join([ch, str(i)]) for ch in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for i in range(1, 13)]

    # Desired order
    desired_order = [
        'A1', 'C1', 'E1', 'G1', 
        'B2', 'D2', 'F2', 'H2',
        'A3', 'C3', 'E3', 'G3',
        'B4', 'D4', 'F4', 'H4',
        'A5', 'C5', 'E5', 'G5', 
        'B6', 'D6', 'F6', 'H6', 
        'A7', 'C7', 'E7', 'G7',
        'B8', 'D8', 'F8', 'H8',
        'A9', 'C9', 'E9', 'G9',
        'B10', 'D10', 'F10', 'H10',
        'A11', 'C11', 'E11', 'G11',
        'B12', 'D12', 'F12', 'H12',
        'B1', 'D1', 'F1', 'H1',
        'A2', 'C2', 'E2', 'G2',
        'B3', 'D3', 'F3', 'H3',
        'A4', 'C4', 'E4', 'G4', 
        'B5', 'D5', 'F5', 'H5',
        'A6', 'C6', 'E6', 'G6',
        'B7', 'D7', 'F7', 'H7', 
        'A8', 'C8', 'E8', 'G8',
        'B9', 'D9', 'F9', 'H9', 
        'A10', 'C10', 'E10', 'G10',
        'B11', 'D11', 'F11', 'H11',
        'A12', 'C12', 'E12', 'G12'
    ]

    # Headers
    headers = [
        'location',
        'stock A (uL)',
        'stock B (uL)',
        'stock C (uL)'
    ]

    # Dispense volumes
    dispense_volumes = []
    with open(filename, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                dispense_volumes.append([int(i) for i in row])

    # Output file
    output_file = f'{os.path.splitext(filename)[0]}._nimbus.xls'

    # Reorder list
    reordered = []
    reordered.append(headers)
    association = named_association(default_well_pos, dispense_volumes)
    for element in desired_order:

        # Lookup value and append
        reordered.append([
            element,
            association[element][0],
            association[element][1],
            association[element][1]
        ])

    # Export
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    for element in reordered:

        sheet.append(element)

    workbook.save(output_file)

### Write to Sidekick file

This file can be sent by serial port to the device

In [19]:
def pump_instruction(position, volumes):
    '''
    Funtion to write pump instructions

    @param position : well positions
    @param volumes : volume list
    '''
    # Create instructions
    instructions = [''.join(['p',f'{str(element)[0]} ', position, f' {str(element)}']) for element in volumes]

    # Return instructions
    return instructions

def convert_CSV_to_Sidekick(filename):
    '''
    Function to convert CSV to Sidekick format

    @param filename : file to read
    '''
    # Well positions
    default_well_pos = [''.join([ch, str(i)]) for ch in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for i in range(1, 13)]

    # Dispense volumes
    dispense_volumes = []
    with open(filename, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                dispense_volumes.append([i for i in row])

    # Output file
    output_file = f'{os.path.splitext(filename)[0]}._sidekick.txt'

    # Create instructions
    commands = [pump_instruction(pos, vol) for pos, vol in zip(default_well_pos, dispense_volumes)]
    commands = sum(commands, [])
    commands.append('home')
    
    # Export commands
    with open(output_file, 'w') as csvfile:
        for row in commands:
            csvfile.write(row)

### Write to Serial Port (for Sidekick)

**Comment:** You can also directly control the sidekick from the serial port. Here's some example code showing how you might do this (We don't do this in the article.)

In [ ]:
def pump(device, position, vA, vB, vC):
    '''
    Function to control sidekick from serial port

    @param device : device to write to
    @param position : well positions
    @param vA, vB, vC : component volumes
    '''
    # Create commands
    commands = [''.join(['p', f'{str(v)[0]} ', position, str(v)]) for v in [vA, vB, vC]] 

    # Write command
    device.write(commands.encode())

def home(device):
    '''
    Function to write home

    @param device : serial to write to
    '''
    device.write('home'.encode())

Open the serial port and do various preparatory work, purging, calibration:

In [ ]:
# Serial port
ser = serial.Serial('/dev/tty...')

Commands to execute

In [ ]:
# Run code
home(ser)
pump(ser, instructions)

When done...

In [ ]:
ser.close()

## File Conversion Process

1. ***Copy this Python notebook to the folder containing the CSV files you wish to convert***, by "File>Save As..." to the desired location.
2. Check that 96 experiments have been provided in the CSV file by ***running the code below***. (When you do this, it will set up some initialization cells defined in the hidden cells of this notebook.) Any outputs below correspond to files that contain have more or fewer reactions than 96 specified. ***Modify the files accordingly until this returns an empty list before proceeding to step #3***.

In [ ]:
notebook_directory = os.path.dirname(os.path.abspath(__file__))
os.chdir(notebook_directory)

3. ***Run the following code cell***. It will look for every file that ends in ".csv" in the location where the notebook is located and generate NIMBUS input files (with the ".xls" file extensions) and Sidekick control instructions (with ".txt" file extensions); only files that contain 96 reactions will be processed.

In [ ]:
for filename in os.listdir(notebook_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(notebook_directory, filename)
        convert_CSV_to_NIMBUS(file_path)

4. Use the files on the robot as desired.